## RBF Kernel Soft-SVM Classifier
The code for this section is written inside svm/classifiers/.

In [3]:
from svm.classifiers.RBF_kernel_svm import KernelSVMClassifier
import time
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

import matplotlib.pyplot as plt

diabetes = pd.read_csv('svm/datasets/diabetes.csv')
x_train, x_test, y_train, y_test = train_test_split(np.array(diabetes.loc[:, diabetes.columns != 'Outcome']), np.array(diabetes['Outcome']), stratify=np.array(diabetes['Outcome']), random_state=66)
print(x_train.shape, x_test.shape)

num_training = 480
num_validation = 96 
num_test = 160
num_dev = 32

# Change label set from {0, 1} to {-1, 1}
y_train = 2 * y_train - 1
y_test = 2 * y_test - 1

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = x_train[mask]
y_val = y_train[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = x_train[mask]
y_train = y_train[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = x_train[mask]
y_dev = y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = x_test[mask]
y_test = y_test[mask]

# generate a test classfier for small dataset
clf_test = KernelSVMClassifier(X_dev, y_dev)
clf_test.update_kernel_matrix()
index = np.random.randint(0, num_dev, 1)
loss, grad = clf_test.RBF_kernel_svm_loss(index, C=1.0)

#print scores.shape
print('loss: %f' % (loss, ))

(576, 8) (192, 8)
loss: 1.008220


### Stochastic Gradient Descent

We now have vectorized and efficient expressions for the loss, the gradient. We are therefore ready to do SGD to minimize the loss of the SVM.

In [4]:
clf = KernelSVMClassifier(X_train, y_train)
tic = time.time()
loss_history = clf.train(learning_rate=1e-7, C=1, num_iters=2000, verbose=False)
toc = time.time()
print('That took %fs' % (toc - tic))
# print('loss history: ')
# print(loss_hist)

That took 0.732497s


### Evaluation

In [5]:
# Evaluate the performance on both the training and validation set

y_train_pred = clf.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))

y_val_pred = clf.predict(X_val)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

training accuracy: 0.660417
validation accuracy: 0.666667


### Hyperparameter Tuning

In [6]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths.

best_val = -1
best_svm = None

# Grid Search
C_list = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000]
rate_list = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
trn_accuracy_val = -1 # highest validation accuracy for training set

best_trn_C = None
best_trn_rate = None
best_val_C = None
best_val_rate = None

for c in C_list:
    for rate in rate_list:
        svm = KernelSVMClassifier(X_train, y_train)
        loss_hist = svm.train(learning_rate=rate, C=c, num_iters=2000, verbose=False)

        y_train_pred = svm.predict(X_train)
        val = np.mean(y_train == y_train_pred)
        if val > trn_accuracy_val:
            trn_accuracy_val = val
            best_trn_C = c
            best_trn_rate = rate
        
        y_val_pred = svm.predict(X_val)
        val = np.mean(y_val == y_val_pred)
        if val > best_val:
            best_val = val
            best_val_C = c
            best_val_rate = rate

# Finer search on C ...
C_step = best_val_C / 10

for i in range(5, 15):
    c = i * C_step
    svm = KernelSVMClassifier(X_train, y_train)
    loss_hist = svm.train(learning_rate=best_val_rate, C=c, num_iters=2000, verbose=False)

    y_val_pred = svm.predict(X_val)
    val = np.mean(y_val == y_val_pred)
    if val > best_val:
        best_val = val
        best_val_C = c
        best_svm = svm

# Print out results.
print('best validation accuracy achieved during cross-validation: %f' % best_val)
print('hyperparameter C when best accuracy occurs: %f' % best_val_C)
print('learning rate when best accuracy occurs: %f' % best_val_rate)

best validation accuracy achieved during cross-validation: 0.729167
hyperparameter C when best accuracy occurs: 0.010000
learning rate when best accuracy occurs: 0.000010


### Test Dataset Evaluation

In [12]:
clf = KernelSVMClassifier(X_train, y_train)
loss_hist = clf.train(learning_rate=best_val_rate, C=best_val_C, num_iters=64000, verbose=False)

y_pred = clf.predict(X_test)
val = np.mean(y_test == y_pred)

print('validation accuracy achieved on test dataset: %f' % val)

validation accuracy achieved on test dataset: 0.687500
